Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import sklearn.ensemble
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import OrdinalEncoder
from lightgbm import LGBMRegressor
from sklearn.linear_model import LassoCV
from sklearn.model_selection import cross_val_score

Importing datasets

In [2]:
data1 = pd.read_csv('C:/KUSHAL/Projects/Kaggle/Rohlik Sales Forecasting/inbox/sales_train.csv', encoding = 'utf-8')
data2 = pd.read_csv('C:/KUSHAL/Projects/Kaggle/Rohlik Sales Forecasting/inbox/sales_test.csv', encoding = 'utf-8')
data3 = pd.read_csv('C:/KUSHAL/Projects/Kaggle/Rohlik Sales Forecasting/inbox/calendar.csv', encoding = 'utf-8')
data4 = pd.read_csv('C:/KUSHAL/Projects/Kaggle/Rohlik Sales Forecasting/inbox/inventory.csv', encoding = 'utf-8')
data5 = pd.read_csv('C:/KUSHAL/Projects/Kaggle/Rohlik Sales Forecasting/inbox/solution.csv', encoding = 'utf-8')
data6 = pd.read_csv('C:/KUSHAL/Projects/Kaggle/Rohlik Sales Forecasting/inbox/test_weights.csv', encoding = 'utf-8')
print("Datasets successfully imported")

Datasets successfully imported


checking dtypes of data1, changing date dtype

In [3]:
print(data1.info())
print(data1.isnull().sum()) #info of train set

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4007419 entries, 0 to 4007418
Data columns (total 14 columns):
 #   Column           Dtype  
---  ------           -----  
 0   unique_id        int64  
 1   date             object 
 2   warehouse        object 
 3   total_orders     float64
 4   sales            float64
 5   sell_price_main  float64
 6   availability     float64
 7   type_0_discount  float64
 8   type_1_discount  float64
 9   type_2_discount  float64
 10  type_3_discount  float64
 11  type_4_discount  float64
 12  type_5_discount  float64
 13  type_6_discount  float64
dtypes: float64(11), int64(1), object(2)
memory usage: 428.0+ MB
None
unique_id           0
date                0
warehouse           0
total_orders       52
sales              52
sell_price_main     0
availability        0
type_0_discount     0
type_1_discount     0
type_2_discount     0
type_3_discount     0
type_4_discount     0
type_5_discount     0
type_6_discount     0
dtype: int64


Finding number of null values and total number of values for data missing unique_ids

In [4]:
unique_ids = [885, 1237, 725, 3778, 5152, 2148, 2424, 3178, 1776, 1689, 612, 2809, 794]
filtered = data1[data1['unique_id'].isin(unique_ids)]
results = []
for unique_id in unique_ids:
    current_filtered = filtered[filtered['unique_id'] == unique_id]
    null_1 = current_filtered['total_orders'].isnull().sum()
    non_null_1 = current_filtered['total_orders'].notnull().sum()
    null_2 = current_filtered['sales'].isnull().sum()
    non_null_2 = current_filtered['sales'].notnull().sum()
    results.append({
        'unique_id': unique_id,
        'Null values in total_orders': null_1,
        'Non-null values in total_orders': non_null_1,
        'Null values in sales': null_2,
        'Non-null values in sales': non_null_2
    })
result_data = pd.DataFrame(results)
print(result_data)

    unique_id  Null values in total_orders  Non-null values in total_orders  \
0         885                            4                              294   
1        1237                            2                              528   
2         725                            2                              890   
3        3778                            7                              697   
4        5152                            4                              730   
5        2148                            7                             1046   
6        2424                            8                             1059   
7        3178                            7                             1048   
8        1776                            1                             1039   
9        1689                            2                              400   
10        612                            2                              369   
11       2809                            4          

In [5]:
clean1 = data1.dropna()
print(clean1.info())

<class 'pandas.core.frame.DataFrame'>
Index: 4007367 entries, 0 to 4007418
Data columns (total 14 columns):
 #   Column           Dtype  
---  ------           -----  
 0   unique_id        int64  
 1   date             object 
 2   warehouse        object 
 3   total_orders     float64
 4   sales            float64
 5   sell_price_main  float64
 6   availability     float64
 7   type_0_discount  float64
 8   type_1_discount  float64
 9   type_2_discount  float64
 10  type_3_discount  float64
 11  type_4_discount  float64
 12  type_5_discount  float64
 13  type_6_discount  float64
dtypes: float64(11), int64(1), object(2)
memory usage: 458.6+ MB
None


Checking dtypes, null values of data2 and changing the dytypes of date column

In [6]:
print(data2.info())
print(data2.isnull().sum()) #info of test set

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47021 entries, 0 to 47020
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   unique_id        47021 non-null  int64  
 1   date             47021 non-null  object 
 2   warehouse        47021 non-null  object 
 3   total_orders     47021 non-null  float64
 4   sell_price_main  47021 non-null  float64
 5   type_0_discount  47021 non-null  float64
 6   type_1_discount  47021 non-null  float64
 7   type_2_discount  47021 non-null  float64
 8   type_3_discount  47021 non-null  float64
 9   type_4_discount  47021 non-null  float64
 10  type_5_discount  47021 non-null  float64
 11  type_6_discount  47021 non-null  float64
dtypes: float64(9), int64(1), object(2)
memory usage: 4.3+ MB
None
unique_id          0
date               0
warehouse          0
total_orders       0
sell_price_main    0
type_0_discount    0
type_1_discount    0
type_2_discount    0
type_3_disco

Checking dtypes, null values of data3 and changing the dytypes of date column

In [7]:
print(data3.info())
print(data3.isnull().sum()) #info of calender set

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23016 entries, 0 to 23015
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   date                    23016 non-null  object
 1   holiday_name            930 non-null    object
 2   holiday                 23016 non-null  int64 
 3   shops_closed            23016 non-null  int64 
 4   winter_school_holidays  23016 non-null  int64 
 5   school_holidays         23016 non-null  int64 
 6   warehouse               23016 non-null  object
dtypes: int64(4), object(3)
memory usage: 1.2+ MB
None
date                          0
holiday_name              22086
holiday                       0
shops_closed                  0
winter_school_holidays        0
school_holidays               0
warehouse                     0
dtype: int64


Checking dtypes and null values in data4

In [8]:
print(data4.info())
print(data4.isnull().sum()) #info of inventory set

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5432 entries, 0 to 5431
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   unique_id            5432 non-null   int64 
 1   product_unique_id    5432 non-null   int64 
 2   name                 5432 non-null   object
 3   L1_category_name_en  5432 non-null   object
 4   L2_category_name_en  5432 non-null   object
 5   L3_category_name_en  5432 non-null   object
 6   L4_category_name_en  5432 non-null   object
 7   warehouse            5432 non-null   object
dtypes: int64(2), object(6)
memory usage: 339.6+ KB
None
unique_id              0
product_unique_id      0
name                   0
L1_category_name_en    0
L2_category_name_en    0
L3_category_name_en    0
L4_category_name_en    0
warehouse              0
dtype: int64


Checking dtypes and null values in data5

In [9]:
print(data5.info())
print(data5.isnull().sum()) #info of solution set

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47021 entries, 0 to 47020
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         47021 non-null  object
 1   sales_hat  47021 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 734.8+ KB
None
id           0
sales_hat    0
dtype: int64


Checking dtypes and null values in data6

In [10]:
print(data6.info())
print(data6.isnull().sum()) #info of test weights set

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5390 entries, 0 to 5389
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   unique_id  5390 non-null   int64  
 1   weight     5390 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 84.3 KB
None
unique_id    0
weight       0
dtype: int64


In [11]:
merge1 = pd.merge(clean1, data6, on='unique_id')
print("Merge successfull")

Merge successfull


In [12]:
merge2 = pd.merge(merge1, data3, on=['date', 'warehouse'])
print("Merge successfull")

Merge successfull


In [13]:
merge3 = pd.merge(merge2, data4, on=['unique_id', 'warehouse'])
print("Merge successfull")

Merge successfull


Spliting date column in merged file for better understanding

In [14]:
merge3['date'] = pd.to_datetime(merge3['date'], errors='coerce')
merge3['day'] = merge3['date'].dt.day
merge3['month'] = merge3['date'].dt.month
merge3['year'] = merge3['date'].dt.year
print("Procedure successfull")

Procedure successfull


In [15]:
print(merge3.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4007367 entries, 0 to 4007366
Data columns (total 29 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   unique_id               int64         
 1   date                    datetime64[ns]
 2   warehouse               object        
 3   total_orders            float64       
 4   sales                   float64       
 5   sell_price_main         float64       
 6   availability            float64       
 7   type_0_discount         float64       
 8   type_1_discount         float64       
 9   type_2_discount         float64       
 10  type_3_discount         float64       
 11  type_4_discount         float64       
 12  type_5_discount         float64       
 13  type_6_discount         float64       
 14  weight                  float64       
 15  holiday_name            object        
 16  holiday                 int64         
 17  shops_closed            int64         
 18  wi

In [16]:
merge4 = pd.merge(data2, data3, on=['date', 'warehouse'])
print("Merge successfull")

Merge successfull


In [17]:
merge5 = pd.merge(merge4, data4, on=['unique_id', 'warehouse'])
print("Merge successfull")

Merge successfull


In [18]:
merge5.insert(4, 'sales', [None] * len(merge5))
print("Column created successfully")

Column created successfully


In [19]:
merge5.insert(6, 'availability', [None] * len(merge5))
print("Column created successfully")

Column created successfully


In [20]:
merge5.insert(14, 'weight', [None] * len(merge5))
print("Column created successfully")

Column created successfully


Spliting date column in merged file for better understanding

In [21]:
merge5['date'] = pd.to_datetime(merge5['date'], errors='coerce')
merge5['day'] = merge5['date'].dt.day
merge5['month'] = merge5['date'].dt.month
merge5['year'] = merge5['date'].dt.year
print("Procedure successfull")

Procedure successfull


In [22]:
print(merge5.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47021 entries, 0 to 47020
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   unique_id               47021 non-null  int64         
 1   date                    47021 non-null  datetime64[ns]
 2   warehouse               47021 non-null  object        
 3   total_orders            47021 non-null  float64       
 4   sales                   0 non-null      object        
 5   sell_price_main         47021 non-null  float64       
 6   availability            0 non-null      object        
 7   type_0_discount         47021 non-null  float64       
 8   type_1_discount         47021 non-null  float64       
 9   type_2_discount         47021 non-null  float64       
 10  type_3_discount         47021 non-null  float64       
 11  type_4_discount         47021 non-null  float64       
 12  type_5_discount         47021 non-null  float6

In [23]:
merge6 = pd.concat([merge3, merge5], axis=0, ignore_index=True)
print(merge6.info())
print(merge6.notnull().sum())

C:\Users\HP\AppData\Local\Temp\ipykernel_18956\2144504375.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merge6 = pd.concat([merge3, merge5], axis=0, ignore_index=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4054388 entries, 0 to 4054387
Data columns (total 29 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   unique_id               int64         
 1   date                    datetime64[ns]
 2   warehouse               object        
 3   total_orders            float64       
 4   sales                   float64       
 5   sell_price_main         float64       
 6   availability            float64       
 7   type_0_discount         float64       
 8   type_1_discount         float64       
 9   type_2_discount         float64       
 10  type_3_discount         float64       
 11  type_4_discount         float64       
 12  type_5_discount         float64       
 13  type_6_discount         float64       
 14  weight                  float64       
 15  holiday_name            object        
 16  holiday                 int64         
 17  shops_closed            int64         
 18  wi

In [24]:
merge3['log_sales'] = np.log(merge3['sales'] + 1)

In [25]:
def make_time_features(merge3):
    merge3['day'] = merge3['date'].apply(lambda x: x.dayofweek)  
    merge3['week'] = merge3['date'].apply(lambda x: x.isocalendar()[1])  
    merge3['month'] = merge3['date'].apply(lambda x: x.month)
    merge3['day_of_week'] = merge3['date'].apply(lambda x: x.strftime('%A'))
    merge3['day_in_month'] = merge3['date'].apply(lambda x: x.day)
    merge3["day_of_year"] = merge3["date"].apply(lambda x: x.timetuple().tm_yday)
    merge3['year'] = merge3['date'].apply(lambda x: x.year)

    merge3['sin_month'] = np.sin(2 * np.pi * merge3['month'] / 12)
    merge3['sin_day'] = np.sin(2 * np.pi * merge3['day'] / 7)
    merge3['sin_week'] = np.sin(2 * np.pi * merge3['week'] / 53)
    merge3['sin_day_year'] = np.sin(2 * np.pi * merge3['day_of_year'] / 365)

    min_date = merge3["date"].min()
    merge3["days_since_start"] = merge3["date"].apply(lambda x: (x - min_date).days)

    merge3['lag_1'] = merge3['sales'].shift(1)
    merge3['lag_7'] = merge3['sales'].shift(7)
    merge3['lag_30'] = merge3['sales'].shift(30)

    merge3['rolling_mean_7'] = merge3['sales'].rolling(window=7).mean()
    merge3['rolling_std_7'] = merge3['sales'].rolling(window=7).std()
    merge3['is_holiday'] = merge3['holiday'].apply(lambda x: 1 if x else 0)
    merge3['is_weekend'] = merge3['day_of_week'].apply(lambda x: 1 if x in [5, 6] else 0)
    merge3 = pd.get_dummies(merge3, columns=['day_of_week'], drop_first=True)   
    return merge3

In [26]:
make_time_features(merge3)
make_time_features(merge5)

unique_id       date warehouse  total_orders sales  sell_price_main  \
0           1226 2024-06-03    Brno_1        8679.0  None            13.13   
1           1226 2024-06-11    Brno_1        8795.0  None            13.13   
2           1226 2024-06-13    Brno_1       10009.0  None            13.13   
3           1226 2024-06-15    Brno_1        8482.0  None            13.13   
4           1226 2024-06-09    Brno_1        8195.0  None            13.13   
...          ...        ...       ...           ...   ...              ...   
47016       4572 2024-06-03  Munich_1        5254.0  None             2.09   
47017       3735 2024-06-04  Prague_1        9698.0  None            11.00   
47018       3735 2024-06-03  Prague_1       10256.0  None            11.00   
47019       2129 2024-06-03    Brno_1        8679.0  None            37.75   
47020       3030 2024-06-03  Prague_2        5705.0  None            90.03   

      availability  type_0_discount  type_1_discount  type_2_discount  ...  \
0             None          0.00000              0.0              0.0  ...   
1             None          0.15873              0.0              0.0  ...   
2             None          0.15873              0.0              0.0  ...   
3             None          0.15873              0.0              0.0  ...   
4             None          0.00000              0.0              0.0  ...   
...            ...              ...              ...              ...  ...   
47016         None          0.00000              0.0              0.0  ...   
47017         None          0.00000              0.0              0.0  ...   
47018         None          0.00000              0.0              0.0  ...   
47019         None          0.00000              0.0              0.0  ...   
47020         None          0.00000              0.0              0.0  ...   

       rolling_mean_7  rolling_std_7  is_holiday  is_weekend  \
0                 NaN            NaN           0           0   
1                 NaN            NaN           0           0   
2                 NaN            NaN           0           0   
3                 NaN            NaN           0           0   
4                 NaN            NaN           0           0   
...               ...            ...         ...         ...   
47016             NaN            NaN           0           0   
47017             NaN            NaN           0           0   
47018             NaN            NaN           0           0   
47019             NaN            NaN           0           0   
47020             NaN            NaN           0           0   

      day_of_week_Monday day_of_week_Saturday  day_of_week_Sunday  \
0                   True                False               False   
1                  False                False               False   
2                  False                False               False   
3                  False                 True               False   
4                  False                False                True   
...                  ...                  ...                 ...   
47016               True                False               False   
47017              False                False               False   
47018               True                False               False   
47019               True                False               False   
47020               True                False               False   

       day_of_week_Thursday  day_of_week_Tuesday  day_of_week_Wednesday  
0                     False                False                  False  
1                     False                 True                  False  
2                      True                False                  False  
3                     False                False                  False  
4                     False                False                  False  
...                     ...                  ...                    ...  
47016            

In [27]:
def assign_around_hol(merge3, original=1):
    merge3["holiday_name"] = merge3["holiday_name"].fillna("")  # Ensure no NaN values are initially
    for warehouse in merge3["warehouse"].unique():
        warehouse_merge3 = merge3[merge3["warehouse"] == warehouse].copy()

        # Shift the 'holiday' column (lag feature calculation)
        warehouse_merge3["holiday"] = -2 * warehouse_merge3["holiday"].shift(-1).fillna(0) + warehouse_merge3["holiday"] + 2 * warehouse_merge3["holiday"].shift(1).fillna(0)

        # Replace empty strings with NaN for 'holiday_name' to clean up data
        warehouse_merge3["holiday_name"] = warehouse_merge3["holiday_name"].replace("", np.nan)

        # Convert 'holiday_name' column to categorical dtype
        warehouse_merge3["holiday_name"] = warehouse_merge3["holiday_name"].astype("category")

        # Fill missing values in 'holiday_name' using forward fill and backward fill
        warehouse_merge3["holiday_name"] = warehouse_merge3["holiday_name"].ffill()  # Forward fill missing values
        warehouse_merge3["holiday_name"] = warehouse_merge3["holiday_name"].bfill()  # Backward fill missing values

        # Replace the updated warehouse data back into the original DataFrame
        merge3.loc[merge3["warehouse"] == warehouse, "holiday_name"] = warehouse_merge3["holiday_name"]

# Apply the function to both datasets
assign_around_hol(merge3)
assign_around_hol(merge5)

C:\Users\HP\AppData\Local\Temp\ipykernel_18956\3616762359.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  warehouse_merge3["holiday_name"] = warehouse_merge3["holiday_name"].replace("", np.nan)
C:\Users\HP\AppData\Local\Temp\ipykernel_18956\3616762359.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  warehouse_merge3["holiday_name"] = warehouse_merge3["holiday_name"].replace("", np.nan)
C:\Users\HP\AppData\Local\Temp\ipykernel_18956\3616762359.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will b

In [28]:
merge3 = merge3.drop(['date'], axis=1)
merge5 = merge5.drop(['date'], axis=1)
print("Column Dropped successfully")

Column Dropped successfully


In [29]:
columns = ["holiday", "shops_closed", 'winter_school_holidays', 'school_holidays']

def make_lag(merge3, original, columns):
    for column in columns:
        for lag in original:
            merge3[f"{column}_{lag}"] = np.nan  
            
    for column in columns:
        for warehouse in merge3["warehouse"].unique():
            warehouse_data = merge3[merge3["warehouse"] == warehouse].copy()
            
            for lag in original:
                if lag == 0:
                    continue
                warehouse_data[f"{column}_{lag}"] = warehouse_data[column].shift(lag)

            merge3.loc[merge3["warehouse"] == warehouse, [f"{column}_{lag}" for lag in original]] = warehouse_data[[f"{column}_{lag}" for lag in original]]   
    return merge3

In [30]:
y = merge3.log_sales
features = ['sin_day', 'sin_month', 'year', 'sin_day_year', 'total_orders', 'sell_price_main', 'availability', "holiday", 'holiday_name', "shops_closed", 'winter_school_holidays', 'school_holidays', 'warehouse', 'weight']
X = merge3[features]
X = pd.get_dummies(X)
scaler = StandardScaler()
X[['total_orders', 'sell_price_main', 'availability', 'weight']] = scaler.fit_transform(X[['total_orders', 'sell_price_main', 'availability', 'weight']])
print(X.isnull().sum())
print(y.isnull().sum())

<IPython.core.display.Javascript object>

sin_day                                                                     0
sin_month                                                                   0
year                                                                        0
sin_day_year                                                                0
total_orders                                                                0
sell_price_main                                                             0
availability                                                                0
holiday                                                                     0
shops_closed                                                                0
winter_school_holidays                                                      0
school_holidays                                                             0
weight                                                                      0
holiday_name_1848 Revolution Memorial Day (Extra holiday)       

In [31]:
drop_columns = ['warehouse', 'day_of_week', 'holiday_name']
encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=np.nan)
merge3[drop_columns] = encoder.fit_transform(merge3[drop_columns])
merge5[drop_columns] = encoder.transform(merge5[drop_columns])
warehouse_decoder = {i: encoder.categories_[0][i] for i in range(len(encoder.categories_[0]))}
print(warehouse_decoder)
merge3['warehouse'] = merge3['warehouse'].astype("category")
merge5['warehouse'] = merge5['warehouse'].astype("category")
merge3['warehouse'] = merge3['warehouse'].cat.add_categories('Unknown')
merge5['warehouse'] = merge5['warehouse'].cat.add_categories('Unknown')
merge5 = merge5.fillna({'warehouse': 'Unknown', 'day_of_week': 'Unknown', 'holiday_name': 'Unknown'})
print("Encoding Successful")

{0: 'Brno_1', 1: 'Budapest_1', 2: 'Frankfurt_1', 3: 'Munich_1', 4: 'Prague_1', 5: 'Prague_2', 6: 'Prague_3'}
Encoding Successful


In [32]:
train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.2, random_state=42)

In [33]:
rf_model = LGBMRegressor(random_state=1, n_jobs=-1)
param_grid = {
    'max_depth': [6, 8, 10, 12],
    'n_estimators': [1000, 1500, 2000],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
    'min_child_samples': [20, 50, 100],
    'lambda_l1': [0, 0.1, 0.5],
    'lambda_l2': [0, 0.1, 0.5]
}
grid_search = GridSearchCV(estimator=rf_model,
                           param_grid=param_grid,
                           cv=3,
                           n_jobs=-1,
                           verbose=1,
                           scoring='neg_mean_absolute_error')
grid_search.fit(train_X, train_y)
print("Best Hyperparameters: ", grid_search.best_params_)
best_rf_model = grid_search.best_estimator_
rf_val_predictions = best_rf_model.predict(val_X)
rf_val_mae = mean_absolute_error(rf_val_predictions, val_y)
print("Validation MAE for Light Gradient Boost Model: {:,.0f}".format(rf_val_mae))

Fitting 3 folds for each of 8748 candidates, totalling 26244 fits



KeyboardInterrupt



In [ ]:
lgb.plot_importance(rf_model_on_full_data, max_num_features=13, importance_type='split')
plt.show()

In [ ]:
scores = cross_val_score(rf_model_on_full_data, train_X, train_y, cv=5, scoring='neg_mean_absolute_error')
print("Cross-validated MAE:", -scores.mean())

In [ ]:
lasso = LassoCV(cv=5)
lasso.fit(train_X, train_y)
selected_features = [f for f, coef in zip(features, lasso.coef_) if coef != 0]
print("Selected Features:", selected_features)

In [ ]:
print('The accuracy of the model is: ', rf_model.score(val_X, val_y)) 
print('The accuracy of the training model is: ', rf_model.score(train_X, train_y)) #177

In [ ]:
features = [col for col in features if col in X.columns and col in merge5.columns]
train_X = pd.get_dummies(X[features])
test_X = pd.get_dummies(merge5[features])
test_X = test_X.reindex(columns=train_X.columns, fill_value=0)
rf_model_on_full_data = LGBMRegressor(random_state=1, n_jobs=-1)
param_grid = {
    'max_depth': [6, 8, 10, 12],
    'n_estimators': [1000, 1500, 2000],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
    'min_child_samples': [20, 50, 100],
    'lambda_l1': [0, 0.1, 0.5],
    'lambda_l2': [0, 0.1, 0.5]
}
grid_search = GridSearchCV(estimator=rf_model_on_full_data,
                           param_grid=param_grid,
                           cv=3,
                           n_jobs=-1,
                           verbose=1,
                           scoring='neg_mean_squared_error')
grid_search.fit(train_X, y)
print("Best Hyperparameters: ", grid_search.best_params_)
best_model = grid_search.best_estimator_
test_preds = best_model.predict(test_X)
predicted_sales_original = np.exp(test_preds) - 1
merge5['predicted_sales'] = predicted_sales_original
print(merge5[['predicted_sales']].head())

In [ ]:
data5['id'] = data5['id']
print("Check the 'id' column:")
print(data5['id'].head(5))
output = pd.DataFrame({'id': data5.id,
                       'sales_hat': test_preds})
output.to_csv('submission.csv', index=False)

In [ ]:
min_length = min(len(val_y), len(test_preds))
mae = mean_absolute_error(val_y[:min_length], test_preds[:min_length])
mse = MSE(val_y[:min_length], test_preds[:min_length])
rmse = np.sqrt(mse)
r2 = r2_score(val_y[:min_length], test_preds[:min_length])

print(f"R²: {r2}")
print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")

In [ ]:
threshold = 0.5
y_pred_class = np.where(test_preds[:min_length] >= threshold, 1, 0)
val_y_class = np.where(val_y[:min_length] >= threshold, 1, 0)
accuracy = accuracy_score(val_y_class, y_pred_class)
f1 = f1_score(val_y_class, y_pred_class, average='weighted')
precision = precision_score(val_y_class, y_pred_class, average='weighted')
recall = recall_score(val_y_class, y_pred_class, average='weighted')
cm = confusion_matrix(val_y_class, y_pred_class)

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"Confusion Matrix:\n{cm}")

In [ ]:
threshold = 0.5
y_pred_class = np.where(test_preds[:min_length] >= threshold, 1, 0)
val_y_class = np.where(val_y[:min_length] >= threshold, 1, 0)
report = classification_report(val_y_class, y_pred_class)
print(f"Classification Report:\n{report}")

feature_importance_data = pd.DataFrame({ 
    'feature': merge3.columns, 
    'importance': random_forest_model.feature_importances_
}).sort_values('importance', ascending=False)
feature_importance_data